<h2>CS 4780/5780 Final Project: </h2>
<h3>Election Result Prediction for US Counties</h3>

Names and NetIDs for your group members: Eric Osband (eo255), Anthony Cuturuffo (acc284), Eddie Freedman (ebf45???)

<h3>Introduction:</h3>

<p> The final project is about conducting a real-world machine learning project on your own, with everything that is involved. Unlike in the programming projects 1-5, where we gave you all the scaffolding and you just filled in the blanks, you now start from scratch. The programming project provide templates for how to do this, and the most recent video lectures summarize some of the tricks you will need (e.g. feature normalization, feature construction). So, this final project brings realism to how you will use machine learning in the real world.  </p>

The task you will work on is forecasting election results. Economic and sociological factors have been widely used when making predictions on the voting results of US elections. Economic and sociological factors vary a lot among counties in the United States. In addition, as you may observe from the election map of recent elections, neighbor counties show similar patterns in terms of the voting results. In this project you will bring the power of machine learning to make predictions for the county-level election results using Economic and sociological factors and the geographic structure of US counties. </p>
<p>

<h3>Your Task:</h3>
Plase read the project description PDF file carefully and make sure you write your code and answers to all the questions in this Jupyter Notebook. Your answers to the questions are a large portion of your grade for this final project. Please import the packages in this notebook and cite any references you used as mentioned in the project description. You need to print this entire Jupyter Notebook as a PDF file and submit to Gradescope and also submit the ipynb runnable version to Canvas for us to run.

<h3>Due Date:</h3>
The final project dataset and template jupyter notebook will be due on <strong>December 15th</strong> . Note that <strong>no late submissions will be accepted</strong>  and you cannot use any of your unused slip days before.
</p>

![image.png; width="100";](attachment:image.png)

<h2>Part 1: Basics</h2><p>

<h3>1.1 Import:</h3><p>
Please import necessary packages to use. Note that learning and using packages are recommended but not required for this project. Some official tutorial for suggested packacges includes:
    
https://scikit-learn.org/stable/tutorial/basic/tutorial.html
    
https://pytorch.org/tutorials/
    
https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html
<p>

In [3]:
import os
import pandas as pd
import numpy as np
# TODO
from sklearn.preprocessing import StandardScaler
import math
import torch

<h3>1.2 Weighted Accuracy:</h3><p>
Since our dataset labels are heavily biased, you need to use the following function to compute weighted accuracy throughout your training and validation process and we use this for testing on Kaggle.
<p>

In [15]:
def weighted_accuracy(pred, true):
    assert(len(pred) == len(true))
    num_labels = len(true)
    num_pos = sum(true)
    num_neg = num_labels - num_pos
    frac_pos = num_pos/num_labels
    weight_pos = 1/frac_pos
    weight_neg = 1/(1-frac_pos)
    num_pos_correct = 0
    num_neg_correct = 0
    for pred_i, true_i in zip(pred, true):
        num_pos_correct += (pred_i == true_i and true_i == 1)
        num_neg_correct += (pred_i == true_i and true_i == 0)
    weighted_accuracy = ((weight_pos * num_pos_correct) 
                         + (weight_neg * num_neg_correct))/((weight_pos * num_pos) + (weight_neg * num_neg))
    return weighted_accuracy

<h2>Part 2: Baseline Solution</h2><p>
Note that your code should be commented well and in part 2.4 you can refer to your comments. (e.g. # Here is SVM, 
# Here is validation for SVM, etc). Also, we recommend that you do not to use 2012 dataset and the graph dataset to reach the baseline accuracy for 68% in this part, a basic solution with only 2016 dataset and reasonable model selection will be enough, it will be great if you explore thee graph and possibly 2012 dataset in Part 3.

<h3>2.1 Preprocessing and Feature Extraction:</h3><p>
Given the training dataset and graph information, you need to correctly preprocess the dataset (e.g. feature normalization). For baseline solution in this part, you might not need to introduce extra features to reach the baseline test accuracy.
<p>

In [16]:
# You may change this but we suggest loading data with the following code and you may need to change
# datatypes and do necessary data transformation after loading the raw data to the dataframe.
dataset_path = "./train_2016.csv"
# df = pd.read_csv(dataset_path, sep=',',header=None, encoding='unicode_escape')

# Chose to include header to remember column identifiers
df = pd.read_csv(dataset_path, sep=',', encoding='unicode_escape')
df.head()

FIPS                County    DEM    GOP MedianIncome  MigraRate  \
0  18019      Clark County, IN  18791  30012       51,837        4.9   
1   6035     Lassen County, CA   2026   6533       49,793      -18.4   
2  40081    Lincoln County, OK   2423  10838       44,914       -1.3   
3  31153      Sarpy County, NE  27704  44649       74,374        9.2   
4  28055  Issaquena County, MS    395    298       26,957      -12.8   

   BirthRate  DeathRate  BachelorRate  UnemploymentRate  
0       12.8       11.0          20.9               4.2  
1        9.2        6.3          12.0               6.9  
2       11.4       11.7          15.1               5.3  
3       14.2        5.0          40.1               2.9  
4        9.8        5.3           6.7              14.0

In [85]:
actual_pred = lambda x : np.sign(x * 2)
def preprocess(raw_df):
    df = raw_df.copy()
    # Create feature representing state number
    # Gets state initials from a county string
    get_state_from_county = lambda county : county[county.index(",") + 2:]
  
    df["state_name"] = df["County"].apply(get_state_from_county)
    states = df["state_name"].unique().tolist()
  
    
    
    #one-hot encode state data
    onehot = pd.get_dummies(df["state_name"], prefix = None)
    df[onehot.columns] = onehot
    
    #create target label
    target = "target" # Percentage DEM vote, range between 0 and 1. Apply actual_pred(df["target"]) to get actual 0-1 prediction
    df[target] = (df["DEM"] / (df["DEM"] + df["GOP"])).astype(float)
    
    # Get rid of all commas in MedianIncome column
    df['MedianIncome']=df['MedianIncome'].str.replace(',','').astype(int)

    # Get rid of county, state_name, DEM and GOP columns
    df = df.drop(columns = ["state_name", "County", "DEM", "GOP", "FIPS"])
    
    y = df["target"].to_numpy()
    df = df.drop(columns = ["target"])
    x = df.to_numpy()
    
    return x,y

In [87]:
x, y = preprocess(df)
sample_size, input_size = x.shape
print(x.shape)

(1555, 56)


In [88]:
x_tensor, y_tensor = torch.from_numpy(x).float(), torch.from_numpy(y).long()
x_tensor = torch.nn.functional.normalize(x_tensor, dim=0)

In [89]:
dataset = torch.utils.data.TensorDataset(x_tensor, y_tensor)
train, test = torch.utils.data.random_split(dataset, [1000, 555])
train_loader = torch.utils.data.DataLoader(train, shuffle=True,batch_size=1)
test_loader = torch.utils.data.DataLoader(test, batch_size=1)


In [103]:
"""
Three step process for machine learning, ignoring a bunch of important things
we talked about in class:

1. Get data
    1.1 Get a Dataset---this is provided by PyTorch
    1.2 Get a DataLoader---this is provided by PyTorch
2. Get a classifier, and other paraphenalia needed for training
    2.1 Create a classifier architecture, and instantiate it
    2.2 Get a criterion (loss function)
    2.3 Get an optimizer (thing that does parameter updates to the classifier)
3. Classify the data in some reasonably intelligent fashion. This will involve:
    3.1 A one_epoch() function that trains the classifier for one epoch---a
        complete iteration over the dataset from (1.1)
    3.2 A loop that calls one_epoch() the number of times we want to run an
        epoch.
4. ...
5. $$$
"""
# Have you ever *not* needed NumPy?
import numpy as np

# We need to import a bunch of things from PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# tqdm gives us nice progress bars, which provide a huge quality-of-life boost,
# so it's worth mentioning here.
from tqdm import tqdm

# 'something_from_torch = something_from_torch.to(device)' puts the thing onto
# the device. If you have CUDA available, this will massively speed
# computations. Note that all inputs to a computation must be on the same
# device, and that things are on the CPU by default.
device = "cuda" if torch.cuda.is_available() else "cpu"

import torchvision
from torchvision import transforms
from torch.utils.data import Subset

# train_dataset = Subset(dataset, range(0, 1000))
# val_dataset = Subset(dataset, range(1000, 1100))
# val_loader = DataLoader(val_dataset, batch_size=2, num_workers=4)
# input_dim, output_dim, num_epochs = input_size, 2, 100

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda t: torch.flatten(t)),
])
dataset = torchvision.datasets.MNIST("./MNIST/",
    train=True, transform=transform, download=True)
train_dataset = Subset(dataset, range(0, 10000))
val_dataset = Subset(dataset, range(10000, 11000))
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=4)
input_dim, output_dim, num_epochs = 784, 10, 100


# Setting a random seed means we can reproduce our results easily. In my
# understanding, this is generally good practice.
np.random.seed(1701)
torch.manual_seed(1701)

# Here I define a bunch of constants that'll be needed later on. I've chosen the
# first two arbitrarily; the last three are set to values that are probably
# decent across a wide range of tasks.
input_dim = 42          # This is the dimensionality of the inputs X
output_dim = 314159     # This is the number of classes in Y
num_workers = 4         # We want to parallelize loading data during training.
                        #   Otherwise, it's possible for moving data to model to
                        #   become a bottleneck in performance!
batch_size = 16         # This is the number of examples (x,y) that are run
                        #   through the neural net simultaneously. Think of it as
                        #   a mid-ground between SGD and GD.
num_epochs = 100        # The number of complete iterations over the dataset
                        #   made during training

save_file = "best_model.pt" # It's helpful to save the best-performing model
                            #   throughout training...

# 1.1---get the Dataset. A Dataset should extend PyTorch's base class, and needs
# to do two things:
# a) implement the __getitem__() method. This takes in an index to the data and
#       returns the x- and y-values of the ith datapoint, often as tensors.
# b) implement the __len__() method. This is just the amount of data in the
#       dataset.
class ToyDataset(Dataset):
    """An example dataset."""
    
    def __init__(self):
        pass
    
    def __getitem__(self, i):
        raise NotImplementedError()
    
    def __len__(self):
        raise NotImplementedError()
        
# train_dataset = ToyDataset()

# 1.2---get a DataLoader. This wraps a Dataset and allows you to iterate through
# it. It's also where you can implement *a lot* of performance optimizations.
# 'shuffle=True' is set by default, but it's worth emphasizing here.
#
# As long as values for 'batch_size' are reasonable (4 <= x << amount of data),
# changing it will mostly change the speed and not performance of the model.
# 'num_workers' should be just under your number of CPUs.
# train_loader = DataLoader(train_dataset,
#     shuffle=True,
#     batch_size=batch_size,
#     num_workers=num_workers)
    
# 2.1---here we define the actual neural net! It should be a class that inherits
# from torch.nn.Module, and has a forward() method that implements the function
#
#       f : X -> Y
#
# Because the neural net is *literally* a function, I'll call its outputs on an
# input 'x' 'fx'.
class NeuralNet(nn.Module):
    
    # As you know, a neural net is composed of layers. The first layer takes in
    # an example x, does a computation on it, and passes the result fx to the
    # second layer, and this process continues.
    #
    # Note that the architecture below illustrates how to build a neural net,
    # but shouldn't work too well otherwise.
    def __init__(self, input_dim, output_dim):
        super(NeuralNet, self).__init__()
        self.f1 = nn.Linear(input_dim , 2)  # The first layer should map from
                                            #   the dimensionality of the input
                                            #   data to another dimensionality.
        self.r1 = nn.ReLU()                 # Activation functions operate
                                            #   element-wise, so they don't need
                                            #   a dimensionality specified, and
                                            #   they don't change the
                                            #   dimensionality of their inputs.
                                            #   Generally every linear layer is
                                            #   followed by an activation
                                            #   function.
        self.fc = nn.Linear(2, output_dim)  # The last layer should map from the
                                            #   last hidden dimensionality to
                                            #   the desired number of classes.
                                            #   At least in computer vision,
                                            #   it's generally called 'fc'.
                                            
    # Don't worry about adding an activation function at the end. It's done
    # automatically in the loss function! Of course, this means the outputs of
    # the model need to be softmaxed before they can be used as probabilities.
    
    # This is the function that implements f : X -> Y. Note that we can expand
    # it as fc(r1(f1(x)))))!
    def forward(self, x):
        fx = self.f1(x)
        fx = self.r1(fx)
        return self.fc(fx)
        
# Instantiate the model! We need to move it to whatever device it'll be running
# on before we instantiate the optimizer in 2.3.
model = NeuralNet(input_dim, output_dim).to(device)

# 2.2---The criterion is the loss function. Normally, you use CrossEntropy for a
# classification task. (Binary classification is a special case where you can
# get away with doing something else like 0-1 loss.)
criterion = nn.CrossEntropyLoss()

# 2.3---The optimizer takes care of the parameter updates via its step() method.
# To make it work, we need to pass in the 'parameters' of the model. Generally,
# you'll choose between Adam and SGD.
optimizer = optim.Adam(model.parameters())

# 3.1---I find it's useful to define a function implementing a single epoch, and
# then call it repeatedly from within a loop to train the neural net.
def one_epoch(model, train_loader, optimizer, criterion):
    """Returns [model] after being trained for one epoch on [data_loader] using
    [optimizer] and [criterion].
    """
    # [data_loader] returns batches of paired x- and y-values when we iterate
    # over it. Because of PyTorch magic, we don't have to worry about the batch
    # size!
    #
    # What is tqdm doing here? When we wrap an iterable in tqdm() during
    # iteration, we get a super nice progress bar!
    for x,y in tqdm(train_loader, desc="Running batches...", leave=False):
        # x is a tensor of dimension (batch_size x n_features), y is a tensor of
        # dimension (batch_size,). If they're not on the same device as [model],
        # they need to be moved there. (Note that it might be smarter to
        # accomplish this through the DataLoader.)
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()   # We only want to accumulate the gradient for
                                #   the current batch. Therefore, we zero out
                                #   the gradient of the model.
        fx = model(x)           # Compute predictions for a batch 'x'
        loss = criterion(fx, y) # Compute the loss function's value on the
                                #   outputs of the model on 'x' and the true
                                #   labels 'y'
        loss.backward()         # Compute the gradients of the model's weights
                                #   with respect to the loss.
        optimizer.step()        # Update parameters by taking an intelligent
                                #   optimizer-determined step against the
                                #   gradient.
        
    # When the epoch is over, return the model.
    return model

# We can keep track of when after an epoch finishes our model has a better
# validation accuracy than anything else using this. We'll also need a function
# to compute validation accuracy!
best_val_acc = float("-inf")

def validate(model, val_loader):
    """Returns the accuracy of [model] on [val_loader]."""
    
    def batch_acc(x, y):
        """Returns the number of correct predictions of [model] on [x] given
        [y].
        """
        # Note that PyTorch can do most things that NumPy can. model(x) returns
        # a batch of predictions as a (batch_size x n_classes) tensor; y is a
        # (n_examples,) tensor of in which the i^th index contains the class of
        # the i^th example, eg. 5.
        #
        # Additionally, [model] is on [device], so you'll need to move [x] to
        # the device prior to computation. To move a tensor [t] to the CPU you
        # can call 't.cpu()'---this might be useful for comparing with [y].
        #
        # Can you fit this all on one line?
        x.cpu()
        return y.eq(model(x)).sum()
    
    # The length of a Dataset is the number of examples in it; the length of a
    # DataLoader is the number of batches. Therefore, to get the accuracy, it's
    # critical to divide by the length of the first and not the second!
    #
    # We can use NumPy here because it's generally (in my experience) faster for
    # things on the CPU than PyTorch.
    return np.sum([batch_acc(x, y) for x,y in val_loader]) / len(val_dataset)

# 3.2---Train the model on the entirety of the data once for every desired
# epoch. tqdm gives a nice progress bar.
for e in tqdm(range(num_epochs), desc="Running epochs..."):
    model = one_epoch(model, train_loader, optimizer, criterion)
    
    # Suppose we wanted to find the validation accuracy after epoch! All we'd
    # need would be a validation DataLoader, and a function to get accuracy
    # from! Putting this under 'with torch.no_grad():' turns off computing
    # gradients, because we don't need them. Moreover, wrapping it in
    # 'model.eval()' and 'model.train()' is important if we do fancy things like
    # dropout.
    with torch.no_grad():
        model.eval()
        val_acc = validate(model, val_loader)
        model.train()
    
    # When in the middle of a tqdm-ed loop, we need to use 'tqdm.write()'
    # instead of 'print()'.
    tqdm.write(f"Epoch {e:3} | validation accuracy: {val_acc}")
    
    # If the validation accuracy is the highest it's ever been, why not save the
    # the model? This is a natural way to figure out the correct number of
    # epochs!
    if val_acc > best_val_acc:
        tqdm.write(f"------Best epoch yet with accuracy {val_acc}. Saved!")
        best_val_acc = val_acc
        
        # Saving a PyTorch model is super easy! 'model.state_dict()' converts
        # all of its weights to a dictionary format. The general format here is
        # 'torch.save(dictionary, file)'. Note that we can put basically
        # anything we want in this dictionary too!
        torch.save(model.state_dict(), save_file)
        
# Now that training is done, let's load the best model so we can do something
# with it. First, we need to instantiate a new model of the same type as the
# old one. Then we can call 'load_state_dict()' on the loaded state_dict.
model = NeuralNet(input_dim, output_dim)
best_val_acc_model_weights = torch.load(save_file)
model.load_state_dict(best_val_acc_model_weights)

################################################################################
### WANT TO SEE THE CODE RUN? PASTE THE FOLLOWING CODE DIRECTLY ABOVE 1.2, THEN
### IMPLEMENT 'batch_acc()'.
### It's way beyond the scope of the course though still far too simple for the
### task it takes on; I use it to validate what I've written elsewhere. Also
### note that it downloads the MNIST (handwritten images) dataset to your
### computer, so beware...though you really can just delete it later.
###
### Aside: you should be able to get at least 95% accuracy on the MNIST dataset
### without trying. This gets up to about 70% accuracy because I deliberately
### wrote the worst neural net I could think of that'd illustrate what I needed
### to show.
################################################################################

# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Lambda(lambda t: torch.flatten(t)),
# ])
# dataset = torchvision.datasets.MNIST("./MNIST/",
#     train=True, transform=transform, download=True)
# train_dataset = Subset(dataset, range(0, 10000))
# val_dataset = Subset(dataset, range(10000, 11000))
# val_loader = DataLoader(val_dataset, batch_size=64, num_workers=4)
# input_dim, output_dim, num_epochs = 784, 10, 100







Running epochs...:   0%|          | 0/100 [00:00<?, ?it/s]





Running batches...:   0%|          | 0/1000 [00:00<?, ?it/s]





Running epochs...:   0%|          | 0/100 [00:00<?, ?it/s]  


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x56 and 42x2)

In [ ]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
        self.layer_out = torch.nn.Linear(num_classes, 1)
        self.sigmoid = torch.nn.Sigmoid()

    
    def forward(self, x):
        out = self.fc1(x)
        #out = self.relu(out)
        out = self.sigmoid(out)

        out = self.fc2(out)
        out = self.sigmoid(out)
        #out = self.relu(out)
        out = self.layer_out(out)
        #out = torch.nn.functional.log_softmax(out)
        #out = self.sigmoid(out)
        return out

In [72]:
num_classes = hidden_size = input_size
model = NeuralNet(input_size, hidden_size, num_classes)
epochs = 5
learning_rate = .001

# Loss and optimizer
def my_loss(output, target):
    loss = torch.mean((10*(output - target))**2)
    return loss
criterion = my_loss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

#Training model
total_step = len(train_loader)
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = torch.autograd.Variable(data), torch.autograd.Variable(target)
        #print(data)
        optimizer.zero_grad()

        net_out = model(data)
        #print(net_out,target)
        loss = criterion(net_out, target)
        
        # Backprop
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 199:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(train_loader.dataset),
                               100. * batch_idx / len(train_loader), loss.data))


Train Epoch: 0 [199/1000 (20%)]	Loss: 16.482794
Train Epoch: 0 [399/1000 (40%)]	Loss: 27.657545
Train Epoch: 0 [599/1000 (60%)]	Loss: 8.843080
Train Epoch: 0 [799/1000 (80%)]	Loss: 0.319080
Train Epoch: 0 [999/1000 (100%)]	Loss: 5.631245
Train Epoch: 1 [199/1000 (20%)]	Loss: 15.564516
Train Epoch: 1 [399/1000 (40%)]	Loss: 28.379908
Train Epoch: 1 [599/1000 (60%)]	Loss: 8.679072
Train Epoch: 1 [799/1000 (80%)]	Loss: 0.284984
Train Epoch: 1 [999/1000 (100%)]	Loss: 5.707983
Train Epoch: 2 [199/1000 (20%)]	Loss: 15.328143
Train Epoch: 2 [399/1000 (40%)]	Loss: 28.585480
Train Epoch: 2 [599/1000 (60%)]	Loss: 8.622234
Train Epoch: 2 [799/1000 (80%)]	Loss: 0.270438
Train Epoch: 2 [999/1000 (100%)]	Loss: 5.742008
Train Epoch: 3 [199/1000 (20%)]	Loss: 15.219230
Train Epoch: 3 [399/1000 (40%)]	Loss: 28.682671
Train Epoch: 3 [599/1000 (60%)]	Loss: 8.593343
Train Epoch: 3 [799/1000 (80%)]	Loss: 0.262439
Train Epoch: 3 [999/1000 (100%)]	Loss: 5.761169
Train Epoch: 4 [199/1000 (20%)]	Loss: 15.156590


In [74]:
#TESTING
# run a test loop
convert_pred = lambda x : (torch.sign(x - 0.5) + 1) / 2
test_loss = 0
correct = 0
preds = np.array([])
for data, target in test_loader:
    data, target = torch.autograd.Variable(data), torch.autograd.Variable(target)
    net_out = model(data)
    # sum up batch loss
    test_loss += criterion(net_out, target)
    pred = convert_pred(net_out.data.max(1)[1])  # get the index of the max log-probability
#     print(net_out, pred, target)
#     print(target.data[0])
#     print(np.sign(0.9))
#     print(pred.numpy())
#     preds.append([pred.numpy()])
    correct += pred.eq(convert_pred(target.data)).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
print('\nWeighted accuracy:', preds)


Test set: Average loss: 7.1631, Accuracy: 478/555 (86%)


Weighted accuracy: []
tensor([[0.1141]], grad_fn=<AddmmBackward>)


<h3>2.2 Use At Least Two Training Algorithms from class:</h3><p>
You need to use at least two training algorithms from class. You can use your code from previous projects or any packages you imported in part 1.1.

In [ ]:
# Make sure you comment your code clearly and you may refer to these comments in the part 2.4
# TODO


<h3>2.3 Training, Validation and Model Selection:</h3><p>
You need to split your data to a training set and validation set or performing a cross-validation for model selection.

In [ ]:
# Make sure you comment your code clearly and you may refer to these comments in the part 2.4
# TODO

<h3>2.4 Explanation in Words:</h3><p>
    You need to answer the following questions in the markdown cell after this cell:

2.4.1 How did you preprocess the dataset and features?

2.4.2 Which two learning methods from class did you choose and why did you made the choices?

2.4.3 How did you do the model selection?

2.4.4 Does the test performance reach a given baseline 68% performanc? (Please include a screenshot of Kaggle Submission)

<h2>Part 3: Creative Solution</h2><p>

<h3>3.1 Open-ended Code:</h3><p>
You may follow the steps in part 2 again but making innovative changes like creating new features, using new training algorithms, etc. Make sure you explain everything clearly in part 3.2. Note that reaching the 75% creative baseline is only a small portion of this part. Any creative ideas will receive most points as long as they are reasonable and clearly explained.

In [ ]:
# Make sure you comment your code clearly and you may refer to these comments in the part 3.2
# TODO

<h3>3.2 Explanation in Words:</h3><p>

You need to answer the following questions in a markdown cell after this cell:

3.2.1 How much did you manage to improve performance on the test set compared to part 2? Did you reach the 75% accuracy for the test in Kaggle? (Please include a screenshot of Kaggle Submission)

3.2.2 Please explain in detail how you achieved this and what you did specifically and why you tried this.

<h2>Part 4: Kaggle Submission</h2><p>
You need to generate a prediction CSV using the following cell from your trained model and submit the direct output of your code to Kaggle. The CSV shall contain TWO column named exactly "FIPS" and "Result" and 1555 total rows excluding the column names, "FIPS" column shall contain FIPS of counties with same order as in the test_2016_no_label.csv while "Result" column shall contain the 0 or 1 prdicaitons for corresponding columns. A sample predication file can be downloaded from Kaggle.

In [ ]:
# TODO

# You may use pandas to generate a dataframe with FIPS and your predictions first 
# and then use to_csv to generate a CSV file.

<h2>Part 5: Resources and Literature Used</h2><p>